In [1]:
import pandas as pd
import os 
import numpy as np

In [2]:
# fixed values
slot_upper_limit = 135

In [3]:
dfa = pd.read_csv("test_data.csv")

In [4]:
dfa.shape

(1040, 5)

In [5]:
dfa.columns

Index(['id', 'first_name', 'last_name', 'email', 'test_slot'], dtype='object')

In [6]:
slot_based = dfa['test_slot'].value_counts()

In [7]:
slot_based

test_slot_7    145
test_slot_4    135
test_slot_2    135
test_slot_1    132
test_slot_5    127
test_slot_6    123
test_slot_3    122
test_slot_8    121
Name: test_slot, dtype: int64

In [8]:
dfa.duplicated(keep=False)

0       False
1       False
2       False
3       False
4       False
        ...  
1035    False
1036    False
1037    False
1038    False
1039    False
Length: 1040, dtype: bool

In [9]:
dfb = dfa.drop_duplicates(subset=['email'], keep='last')

In [10]:
dfb.shape

(1000, 5)

In [11]:
slot_based = dfb['test_slot'].value_counts()

In [12]:
slot_based

test_slot_7    138
test_slot_4    128
test_slot_2    127
test_slot_1    127
test_slot_5    124
test_slot_8    120
test_slot_6    120
test_slot_3    116
Name: test_slot, dtype: int64

In [13]:


line1 = "total registrations with duplicates " + str(dfa.shape[0]) + "\n"
line2 = "total registrations without duplicates " + str(dfb.shape[0]) + "\n"

In [14]:
line1

'total registrations with duplicates 1040\n'

In [15]:
line3 = "total slots filled (with duplicates) : " + "\n"

In [16]:
total_slots = len(dfb['test_slot'].value_counts()) 

In [17]:
counts = str(slot_based)

In [18]:
line4 = counts[0:counts.rfind('\n')] + "\n"

In [19]:
print(line4)

test_slot_7    138
test_slot_4    128
test_slot_2    127
test_slot_1    127
test_slot_5    124
test_slot_8    120
test_slot_6    120
test_slot_3    116



In [20]:
from datetime import datetime

now = datetime.now()

line0 = now.strftime("on %a, %d %b at %I:%M:%p\n")

print(line0)

current_time = now.strftime("%d_%b_%I_%M_%p")

print(current_time)

on Thu, 08 Sep at 01:18:AM

08_Sep_01_18_AM


In [21]:
file_name = "status_" + current_time + ".txt"

In [22]:
with open(file_name, "w") as fa:
    fa.write(line0)
    fa.write(line1)
    fa.write(line2)
    fa.write(line3)
    fa.write(line4)

In [23]:
slot_dict = dict(slot_based)

In [24]:
slot_dict

{'test_slot_7': 138,
 'test_slot_4': 128,
 'test_slot_2': 127,
 'test_slot_1': 127,
 'test_slot_5': 124,
 'test_slot_8': 120,
 'test_slot_6': 120,
 'test_slot_3': 116}

In [25]:
count = 0
slot_overflow = ""

for k, v in slot_dict.items():
    if v > slot_upper_limit:
        count = count + 1
        slot_overflow = slot_overflow + "\n" + k + " has reached " + str(v) + " bookings "
        
if count > 0:
    critical = "needs attention, slots are filling up\n"
    critical_msg = "Report : needs attention"
    slot_overflow = f"The following slots have crossed {slot_upper_limit}" + slot_overflow
else:
    critical = f"no slots above {slot_upper_limit} yet\n"
    critical_msg = "Report: all good"

In [26]:
print(slot_overflow)

The following slots have crossed 135
test_slot_7 has reached 138 bookings 


In [27]:
print(critical)

needs attention, slots are filling up



In [29]:
print(critical_msg)

Report : needs attention


In [30]:
import os

In [31]:
help(os.stat)

Help on built-in function stat in module posix:

stat(path, *, dir_fd=None, follow_symlinks=True)
    Perform a stat system call on the given path.
    
      path
        Path to be examined; can be string, bytes, a path-like object or
        open-file-descriptor int.
      dir_fd
        If not None, it should be a file descriptor open to a directory,
        and path should be a relative string; path will then be relative to
        that directory.
      follow_symlinks
        If False, and the last element of the path is a symbolic link,
        stat will examine the symbolic link itself instead of the file
        the link points to.
    
    dir_fd and follow_symlinks may not be implemented
      on your platform.  If they are unavailable, using them will raise a
      NotImplementedError.
    
    It's an error to use dir_fd or follow_symlinks when specifying path as
      an open file descriptor.



In [ ]:
import pandas as pd
import os
import numpy as np
from datetime import datetime
import os
import sys
import time
import smtplib, ssl
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication

# fixed values
slot_upper_limit = 140
data_file = "test_data.csv"
count = 0
slot_overflow = ""
cached_time_stamp = os.stat(data_file).st_mtime
frequency = 1
smtp_server = 'smtp.gmail.com'
smtp_port = 587
sender_id = "trainer.nigam@gmail.com"
password = "iwqxwpojganikbbl"
email_line = "please check the attached text file for details"
critical_msg = ""
critical = ""


def check_slots(df):
    global count
    global slot_overflow
    global critical_msg
    global critical
    dfa = pd.read_csv(df)
    slot_based = dfa['test_slot'].value_counts()
    dfb = dfa.drop_duplicates(subset=['email'], keep='last')

    line1 = "total registrations with duplicates " + str(dfa.shape[0]) + "\n"
    line2 = "total registrations without duplicates " + str(dfb.shape[0]) + "\n"
    line3 = "total slots filled (with duplicates) : " + "\n"
    total_slots = len(dfb['test_slot'].value_counts())
    counts = str(slot_based)
    line4 = counts[0:counts.rfind('\n')] + "\n"

    now = datetime.now()
    line0 = now.strftime("on %a, %d %b at %I:%M:%p\n")
    current_time = now.strftime("%d_%b_%I_%M_%p")
    file_name = "status_" + current_time + ".txt"

    with open(file_name, "w") as fa:
        fa.write(line0)
        fa.write(line1)
        fa.write(line2)
        fa.write(line3)
        fa.write(line4)

    slot_dict = dict(slot_based)

    for k, v in slot_dict.items():
        if v > slot_upper_limit:
            count = count + 1
            slot_overflow = slot_overflow + "\n" + k + " has reached " + str(v) + " bookings "

    if count > 0:
        critical = "needs attention, slots are filling up\n"
        critical_msg = "Report : needs attention"
        slot_overflow = f"The following slots have crossed {slot_upper_limit}" + slot_overflow
    else:
        critical = f"no slots above {slot_upper_limit} yet\n"
        critical_msg = "Report: all good"

    print("intiating email")
    message = MIMEMultipart('mixed')
    message['From'] = 'Report'.format(sender_id)
    message['To'] = 'mr.nigam@gmail.com'
    message['CC'] = 'sukhb9119@gmail.com'
    message['Subject'] = critical_msg

    msg_content = critical + line2 + slot_overflow + email_line
    body = MIMEText(msg_content, 'plain')
    message.attach(body)

    fa = open(file_name)
    attachment = MIMEText(fa.read())
    attachment.add_header('Content-disposition', 'attachment', filename=file_name)
    message.attach(attachment)
    msg_full = message.as_string()

    context = ssl.create_default_context()
    with smtplib.SMTP(smtp_server, smtp_port) as server:
        print("entered ttls")
        server.ehlo()
        server.starttls(context=context)
        server.login(sender_id, password)
        server.sendmail(message["From"], message["To"].split(","), message["CC"].split(","), msg_full)
        server.quit()
    print("send the email successfully")


def check_time_stamp(df):
    global cached_time_stamp
    stamp = os.stat(df).st_mtime
    if stamp != cached_time_stamp:
        cached_time_stamp = stamp
        print("file has changed, intiating check_slots function")
        check_slots(data_file)

def watch(df):
    while True:
        try:
            time.sleep(frequency)
            check_time_stamp(df)
        except FileNotFoundError:
            pass
        except KeyboardInterrupt:
            print("user has shut down the process")
            os._exit(0)

def send_email():
    message = MIMEMultipart('mixed')
    message['From'] = 'Report'.format(sender_id)
    message['To'] = 'mr.nigam@gmail.com'
    message['CC'] = 'sukhb9119@gmail.com'
    message['Subject'] = critical_msg

    msg_content = critical + line2 + slot_overflow + email_line
    body = MIMEText(msg_content, 'plain')
    message.attach(body)

    fa = open(file_name)
    attachment = MIMEText(fa.read())
    attachment.add_header('Content-disposition', 'attachment', filename=file_name)
    message.attach(attachment)
    msg_full = message.as_string()

    context = ssl.create_default_context()
    with smtplib.SMTP(smtp_server, smtp_port) as server:
        server.ehlo()
        server.starttls(context=context)
        server.ehlo()
        server.login(sender_id, password)
        server.sendmail(message["From"], message["To"].split(","), message["Cc"].split(","), msg_full)
        server.quit()
    print("send the email successfully")

watch(data_file)


file has changed, intiating check_slots function
intiating email
entered ttls
